#  CAMEL Cookbook with human-in-loop and tool approval from HumanLayer

You can also check this cookbook in colab [here](https://colab.research.google.com/drive/1WF1Z6Ev6kTrifRLXXTTOZz6-QVRuj1uX?usp=sharing)  (Use the colab share link)

This notebook demonstrates how to set up and leverage CAMEL's ability to interact with user (for approval or comments) during the execution of the tasks.

In this notebook, you'll explore:

*   **CAMEL**: A powerful multi-agent framework that enables Retrieval-Augmented Generation and multi-agent role-playing scenarios, allowing for sophisticated AI-driven tasks.
*   **HumanLayer**: HumanLayer is an API and SDK that enables AI Agents to contact humans for feedback, input, and approvals.
*   **Human-in-loop**: The ability for agent to consult human during the execution of the task.
*   **Human approval**: The ability for agent ask approval to execute some tasks.

This cookbook demonstrates how to use **HumanLayer** functionality within Camel framework.

⭐ **Star the Repo**

If you find CAMEL useful or interesting, please consider giving it a star on our [CAMEL GitHub Repo](https://github.com/camel-ai/camel)! Your stars help others find this project and motivate us to continue improving it.

## 📦 Installation

First, install the CAMEL package with all its dependencies:

In [ ]:
!pip install "camel-ai[all]==0.2.16"

Next, install humanlayer python SDK:

In [ ]:
!pip install humanlayer

## 🔑 Setting Up API Keys

We will need to register HumanLayer to get the API key. For the agent backend LLM model, here we use OpenAI GPT-4o as example:

In [ ]:
# Prompt for the API key securely
import os
from getpass import getpass
openai_api_key = getpass('Enter your OpenAI API key: ')
os.environ["OPENAI_API_KEY"] = openai_api_key
humanlayer_api_key = getpass('Enter your HumanLayer API key: ')
os.environ["HUMANLAYER_API_KEY"] = humanlayer_api_key


Enter your OpenAI API key: ··········
Enter your HumanLayer API key: ··········


## 👨 Tools that requires human approval

In this section, we'll demonstrate how to define tools for Camel agent to use, and use **HumanLayer** to make some tools require human approval. First define two functions for agent to use, one of them requires human approval.

In [ ]:
from humanlayer.core.approval import HumanLayer
hl = HumanLayer(api_key=humanlayer_api_key, verbose=True)


# add can be called without approval
def add(x: int, y: int) -> int:
    """Add two numbers together."""
    return x + y


# but multiply must be approved by a human
@hl.require_approval()
def multiply(x: int, y: int) -> int:
    """multiply two numbers"""
    return x * y

/usr/local/lib/python3.10/dist-packages/pydantic/main.py:214: UserWarning: A custom validator is returning a value other than `self`.
Returning anything other than `self` from a top level model validator isn't supported when validating via `__init__`.
See the `model_validator` docs (https://docs.pydantic.dev/latest/concepts/validators/#model-validators) for more details.
  warnings.warn(


Next we define the Camel agents, then run the computation commands. Here we will need to login HumanLayer cloud platform to approve for the agent to use the multiply function.

In [ ]:
from camel.toolkits import FunctionTool
from camel.agents import ChatAgent
from camel.models import ModelFactory
from camel.types import ModelPlatformType, ModelType

model = ModelFactory.create(
    model_platform=ModelPlatformType.OPENAI,
    model_type=ModelType.GPT_4O,
)

tools = [FunctionTool(add), FunctionTool(multiply)]

agent_with_tools = ChatAgent(
    model = model,
    tools=tools
)

# Interact with the agent
response = agent_with_tools.step("multiply 2 and 5, then add 32 to the result")
print("\n\n----------Result----------\n\n")
print(response.msgs[0].content)


HumanLayer: waiting for approval for multiply via humanlayer cloud
HumanLayer: human approved multiply


----------Result----------


The result of multiplying 2 and 5, then adding 32 to the result, is 42.


## 🤖 Human-in-loop interaction

Sometimes we want the agent to ask user during the working process, in this case, we can equip agent with human toolkits, and be able to ask human via console. This example demonstrates the human-in-loop function:

In [ ]:
from camel.toolkits import HumanToolkit
human_toolkit = HumanToolkit()

agent = ChatAgent(
    system_message="You are a helpful assistant.",
    model=model,
    tools=[*human_toolkit.get_tools()],
)

response = agent.step(
    "Tell me some jokes about monkey, and ask me for feedback."
)

print(response.msgs[0].content)

Question: Why don't monkeys use pocket watches? Because they can't find the right banana hand!
Your reply: Not so funny, change another one
Question: What do you call a monkey that loves to eat potato chips? A chipmunk!
Your reply: Haha, you got me!
I'm glad you liked that one! If you want to hear more monkey jokes, just let me know!


## 🌟 Highlights

This notebook has guided you through setting up chat agents with the ability of **Human-in-loop** and **Human approval**.

Key tools utilized in this notebook include:

*   **CAMEL**: A powerful multi-agent framework that enables Retrieval-Augmented Generation and multi-agent role-playing scenarios, allowing for sophisticated AI-driven tasks.
*   **HumanLayer**: HumanLayer is an API and SDK that enables AI Agents to contact humans for feedback, input, and approvals.
*   **Human-in-loop**: The ability for agent to consult human during the execution of the task.
*   **Human approval**: The ability for agent ask approval to execute some tasks.


⭐ **Star the Repo**

If you find CAMEL useful or interesting, please consider giving it a star on [GitHub](https://github.com/camel-ai/camel)! Your stars help others find this project and motivate us to continue improving it.